In [72]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

import chromadb 
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.core import load_index_from_storage
from llama_index.core import Document
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import FunctionTool, QueryEngineTool
from llama_index.core.agent import ReActAgent

from sklearn.metrics.pairwise import cosine_similarity

from dotenv import load_dotenv, find_dotenv 
import os 
from llama_index.llms.groq import Groq 

import gradio as gr
import pandas as pd 
from pathlib import Path

import inspect 

In [ ]:
def consulta_nota(params: dict):
    caminho_csv = "Notas_alunos.csv"
    df = pd.read_csv(caminho_csv)

    

In [169]:
parametros_usados = {} 
def consultar_notas(params: dict):
    """
    params: dict com chaves opcionais:
        turma, ano_letivo, matriculas (lista), alunos (lista),
        disciplina, bimestre (string "1", "2", "3", "4"),
        nota_maxima (float)
    """
    caminho_csv = "Notas_alunos.csv"
    df = pd.read_csv(caminho_csv)

    turma = params.get("turma")
    ano_letivo = params.get("ano_letivo")
    matriculas = params.get("matriculas")
    alunos = params.get("alunos") or params.get("aluno")
    disciplina = params.get("disciplina")
    bimestre = params.get("bimestre")
    nota_maxima = params.get("nota_maxima", 7.0)

    df_filtrado = df

    if turma:
        parametros_usados["turma"] = turma
        df_filtrado = df_filtrado[df_filtrado['turma'].str.lower() == turma.lower()]
    if ano_letivo:
        parametros_usados["ano_letivo"] = ano_letivo
        df_filtrado = df_filtrado[df_filtrado['ano_letivo'] == int(ano_letivo)]
    if matriculas:
        parametros_usados["matriculas"] = matriculas
        df_filtrado = df_filtrado[df_filtrado['matricula'].isin(matriculas)]
    if alunos:
        parametros_usados["alunos"] = alunos
        df_filtrado = df_filtrado[
            df_filtrado['aluno'].apply(lambda x: any(a.lower() in x.lower() for a in alunos))
        ]
    if disciplina:
        parametros_usados["disciplina"] = disciplina
        df_filtrado = df_filtrado[df_filtrado['disciplina'].str.lower() == disciplina.lower()]
    if bimestre:
        coluna_nota = f'nota_{bimestre}bim'
        parametros_usados["bimestre"] = bimestre
        #parametros_usados["nota_maxima"] = nota_maxima

        if coluna_nota not in df_filtrado.columns:
            return "Bimestre inválido."
        df_filtrado = df_filtrado[df_filtrado[coluna_nota] < float(nota_maxima)]

    if df_filtrado.empty:
        return f"Nenhum aluno encontrado com os filtros aplicados: {parametros_usados}"

    respostas = []
    for _, row in df_filtrado.iterrows():
        nota = row.get(f'nota_{bimestre}bim') if bimestre else "N/A"
        respostas.append(
            f"{row['aluno']} (Matrícula: {row['matricula']}) - Nota: {nota} no {bimestre}º bimestre - Turma: {row['turma']}"
        )

    nota = df_filtrado.iloc[0][coluna_nota]
    parametros_usados["nota"] = str(nota)
    
    return "\n".join(respostas)

In [74]:
df = pd.read_csv("aulas_todas_materias.csv")
documents = []

for _, row in df.iterrows():
    content = f"Aula: {row['data']}\n\n{row['texto']}" 
    metadata = {
        'data': row['data'],
        'ano': row['ano'],
        'bimestre': f"{row['bimestre']}° bimestre",
        'materia': row['materia'].lower()
    }
    doc = Document(text=content, metadata=metadata)
    documents.append(doc)

documents



[Document(id_='339699c8-66f0-45f9-b5a8-422954fdf413', embedding=None, metadata={'data': '2 ABR', 'ano': 2024, 'bimestre': '2° bimestre', 'materia': 'matematica'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Aula: 2 ABR\n\n2 ABR\r\nOBJETIVOS DE APRENDIZAGEM \r\nDO DC-GOEM - (GO-\r\nEMMAT501A) Compreender o \r\nconceito de função polinomial do 1º\r\ngrau, identificando a relação entre \r\nduas variáveis apresentadas em\r\ntextos de origem socioeconômicas \r\ne/ou de natureza técnico ou\r\ncientífica, entre outros para resolver \r\nsituações problemas do cotidiano.\r\nOBJETOS DE CONHECIMENTO\r\nDO DC-GOEM - Funções\r\npolinomiais do 1º\r\ngrau (função afim,\r\nfunção linear,\r\nfunção constante,\r\nfunção identidade)\r\nAplicação de Atividades via \r\nrecursos digitais\r\nRealização de Atividades via recursos \r\ndigitais', path=N

In [75]:
print(documents[0].get_metadata_str())

data: 2 ABR
ano: 2024
bimestre: 2° bimestre
materia: matematica


### Documentos em aprtes menores

In [76]:
node_parser = SentenceSplitter(chunk_size=1000)
nodes = node_parser.get_nodes_from_documents(documents, show_progress=True)

Parsing nodes: 100%|██████████| 295/295 [00:00<00:00, 513.83it/s]


In [77]:
nodes

[TextNode(id_='62e8ea31-19aa-41f4-80ec-8485aa54bc1b', embedding=None, metadata={'data': '2 ABR', 'ano': 2024, 'bimestre': '2° bimestre', 'materia': 'matematica'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='339699c8-66f0-45f9-b5a8-422954fdf413', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'data': '2 ABR', 'ano': 2024, 'bimestre': '2° bimestre', 'materia': 'matematica'}, hash='c9b4d1899525f9df2829c9d58106af4ede6ffcc8627a26a23a1ba5e77b2f8449')}, metadata_template='{key}: {value}', metadata_separator='\n', text='Aula: 2 ABR\n\n2 ABR\r\nOBJETIVOS DE APRENDIZAGEM \r\nDO DC-GOEM - (GO-\r\nEMMAT501A) Compreender o \r\nconceito de função polinomial do 1º\r\ngrau, identificando a relação entre \r\nduas variáveis apresentadas em\r\ntextos de origem socioeconômicas \r\ne/ou de natureza técnico ou\r\ncientífica, entre outros para resolver \r\nsituações problemas do cotidiano.\r\nOBJETOS DE CONHECIMENTO\

### Gerando Embeddings

In [78]:
class ChromaEmbeddingWrapper:
    def __init__(self, model_name):
        self.model = HuggingFaceEmbedding(model_name=model_name)
        self.name = model_name
    
    def __call__(self, texts):
        return self.model.encode(texts, show_progress_bar=False).tolist()

In [79]:
current_file = Path().resolve()
model_folder = current_file / "all-MiniLM-L6-v2"
str(model_folder)

'C:\\Users\\user\\Documents\\projetos\\python\\tcc\\projeto\\all-MiniLM-L6-v2'

In [80]:
chroma_client = chromadb.Client()

In [81]:
try:
    chroma_collection = chroma_client.get_or_create_collection("doccuments_llm")
except Exception as e:
    print(f"Erro ao carregar ou criar embeddings {e}")

### Salvando Embeddings no DB

In [82]:
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [83]:
embed_model = HuggingFaceEmbedding(model_name=str(model_folder))

In [84]:
index = VectorStoreIndex(nodes, storage_context=storage_context, embed_model=embed_model)

### Verificar conteúdos semelhantes entre bimestres

In [ ]:
def get_text_for_bimestre(bimestre, materia, query_engine):
    prompt = f"Conteúdo do {bimestre}º bimestre de {materia}"
    response = query_engine.query(prompt)
    if hasattr(response, "response"):
        return response.response
    elif isinstance(response, str):
        return response
    else:
        return None
    
def get_embedding(text, embed_model):
    return embed_model([text])[0]


def check_revision_needed(bimestre_atual, materia, query_engine, embed_model, threshold=0.7):
    if bimestre_atual <= 1:
        return False
    
    texto_atual = get_text_for_bimestre(bimestre_atual, materia, query_engine)
    texto_anterior = get_text_for_bimestre(bimestre_atual - 1, materia, query_engine)

    if not texto_atual or not texto_anterior:
        return False
    
    emb_atual = get_embedding(texto_atual, embed_model)  # usa a função com embed_documents
    emb_ant = get_embedding(texto_anterior, embed_model)
    
    sim = cosine_similarity([emb_atual], [emb_ant])[0][0]
    print(f"Similaridade entre bimestre {bimestre_atual} e {bimestre_atual - 1} para {materia}: {sim:.4f}")
    
    return sim >= threshold


### Recuperação de informações

In [85]:
load_dotenv(find_dotenv())

True

In [86]:
GROQ_API = os.environ.get("GROQ_API")

In [87]:
llm = Groq(
    model="llama3-70b-8192",
    api_key=GROQ_API
)

In [88]:
query_engine = index.as_query_engine(llm=llm, similarity_top_k=6)

#### Testando o modelo

In [89]:
aluno = "João da Silva"
nota = 4.5
bimestre = 1
materia = "matematica"

def gerar_plano_estudo(aluno, nota, bimestre, materia, query_engine=None):
    if nota >= 8:
        return f"O aluno {aluno} teve nota {nota} no {bimestre}º bimestre. Nenhum plano de estudo é necessário."

    # Filtra os documentos do bimestre e matéria corretos
    docs_filtrados = [
        doc for doc in documents
        if doc.metadata['bimestre'] == f"{bimestre}° bimestre" and doc.metadata['materia'] == materia.lower()
    ]
    if not docs_filtrados:
        return "Nenhum conteúdo encontrado para esse bimestre/matéria."

    # Cria novo index e query_engine só com os docs filtrados
    nodes_filtrados = node_parser.get_nodes_from_documents(docs_filtrados )
    index_filtrado = VectorStoreIndex(nodes_filtrados, embed_model=embed_model)
    query_engine_filtrado = index_filtrado.as_query_engine(llm=llm, similarity_top_k=6)

    prompt = f"""
    O aluno {aluno} obteve nota {nota} no {bimestre}º bimestre na matéria de {materia}.

    Você é um tutor inteligente. Gere um plano de estudo **EXCLUSIVAMENTE** com base no conteúdo de {materia.upper()} do **{bimestre}º bimestre**.

    Ignore completamente conteúdos de outros bimestres, mesmo que relacionados.
    Use **apenas** o que foi ensinado no {bimestre}º bimestre da matéria {materia.upper()}.

    Com base nisso, crie:
    - Conteúdos principais a revisar
    - Dicas de como estudar
    - Sugestões de atividades práticas

    Tudo em português e focado no nível do aluno com nota {nota} e ignora a data que o texto tem foca apenas no bimestre.
    """

    resp = query_engine_filtrado.query(prompt)
    return resp

In [ ]:
gerar_plano_estudo(aluno="pedro", nota=6.5, bimestre=2, materia="Matematica", query_engine=query_engine)

In [180]:
import gradio as gr

def interface_plano_com_feedback(aluno, nota, bimestre, materia):
    try:
        plano = gerar_plano_estudo(aluno, nota, bimestre, materia, query_engine)
        return plano
    except Exception as e:
        return f"Erro: {str(e)}"

with gr.Blocks() as demo:
    aluno = gr.Textbox(label="Nome do aluno")
    nota = gr.Number(label="Nota do aluno")
    bimestre = gr.Number(label="Bimestre")
    materia = gr.Textbox(label="Matéria")

    btn_gerar = gr.Button("Gerar Plano de Estudo")
    saida_plano = gr.Textbox(label="Plano de Estudo", lines=10)

    btn_gerar.click(interface_plano_com_feedback, inputs=[aluno, nota, bimestre, materia], outputs=saida_plano)

demo.launch(share=True)


* Running on local URL:  http://127.0.0.1:7884

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


2025/07/24 17:05:28 [W] [service.go:132] login to server failed: dial tcp 44.237.78.176:7000: i/o timeout


In [170]:
nota_tool = FunctionTool.from_defaults(
    fn=consultar_notas,
    name="ConsultarNotas",
    description="Consulta alunos com nota abaixo de um limite filtrando por turma, disciplina, bimestre, etc. Recebe os filtros como dicionário."
)


In [61]:
retriever = index.as_retriever(similarity_top_k=6)
retriever_tool = QueryEngineTool.from_defaults(
    #query_engine=retriever,
    query_engine=index.as_query_engine(llm=llm, similarity_top_k=6),
    name="Planos de aula",
    description="Ferramenta que busca planos de aula, conteúdos pedagógicos e atividades relacionados à disciplina, bimestre, turma ou aluno. Use esta ferramenta para qualquer consulta sobre plano de ensino, conteúdo, planejamento, atividades ou reforço escolar.",
)

In [28]:
retriever_tool.query_engine.query("quero um plano de aula para um aluno que tem nota baixa em matematica do 1 bimestre fale em portugues")

Response(response='Para um aluno com nota baixa em Matemática no 1° bimestre, é fundamental reforçar os conceitos básicos e trabalhar em atividades que ajudem a consolidar o conhecimento. Aqui vai um plano de aula personalizado:\n\n**Objetivos de Aprendizagem:**\n\n* Revisar e consolidar os conceitos de interpretação de dados e informações em Matemática;\n* Desenvolver habilidades em análise de gráficos e expressões algébricas;\n* Melhorar a compreensão de funções e relações em Matemática.\n\n**Atividades:**\n\n1. Revisão de conceitos: Realizar uma revisão dos conceitos básicos de Matemática, como interpretação de dados, análise de gráficos e expressões algébricas.\n2. Análise de gráficos: Propor atividades que envolvam a análise de gráficos, como identificar padrões, calcular taxas de variação e interpretar resultados.\n3. Exercícios de fixação: Propor exercícios que ajudem a fixar os conceitos, como resolver problemas que envolvam funções e relações.\n4. Atividades práticas: Realizar

In [171]:


chat_engine = ReActAgent.from_tools(
    tools=[nota_tool],
    llm=llm,
    verbose=True
)
#index.as_query_engine(llm=llm,)


c:\Users\user\Documents\projetos\python\tcc\projeto\venv\Lib\site-packages\llama_index\core\agent\react\base.py:154: DeprecationWarning: Call to deprecated class ReActAgent. (ReActAgent has been rewritten and replaced by llama_index.core.agent.workflow.ReActAgent.

This implementation will be removed in a v0.13.0 and the new implementation will be promoted to the `from llama_index.core.agent import ReActAgent` path.

See the docs for more information: https://docs.llamaindex.ai/en/stable/understanding/agent/)
  return cls(
c:\Users\user\Documents\projetos\python\tcc\projeto\venv\Lib\site-packages\deprecated\classic.py:184: DeprecationWarning: Call to deprecated class AgentRunner. (AgentRunner has been deprecated and is not maintained.

This implementation will be removed in a v0.13.0.

See the docs for more information on updated agent usage: https://docs.llamaindex.ai/en/stable/understanding/agent/)
  return old_new1(cls, *args, **kwargs)


In [183]:
def responder(mensagem, chat_history):
    if "plano de aula" in mensagem:
        #def gerar_plano_estudo(aluno, nota, bimestre, materia, query_engine=None):
        #parametros_usados()
        aluno = parametros_usados.get("aluno") or parametros_usados.get("aluno")
        nota = float(parametros_usados.get("nota"))
        bimestre = parametros_usados.get("bimestre")
        materia = parametros_usados.get("materia")
        gerar_plano_estudo(aluno, nota, bimestre, materia)
        #{'turma': '1A', 'bimestre': '2', 'nota': '6.1'}'''
    else:
        resposta = chat_engine.chat(mensagem).response

    if chat_history is None:
        chat_history = []

    chat_history.append(("Você", mensagem))
    chat_history.append(("Assistente", resposta))
    
    return chat_history, chat_history

In [184]:
import gradio as gr

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    entrada = gr.Textbox(placeholder="Digite sua pergunta ou comando...")
    estado = gr.State([])


    entrada.submit(responder, inputs=[entrada, estado], outputs=[chatbot, estado])
    entrada.submit(lambda: "", None, entrada)

demo.launch()

C:\Users\user\AppData\Local\Temp\ipykernel_20032\2862595693.py:4: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


* Running on local URL:  http://127.0.0.1:7886
* To create a public link, set `share=True` in `launch()`.


> Running step 1ff06174-8bea-472b-afb7-bb7317550d78. Step input: quero a nota do aluno ANNA CLARA CALERO GUIDA do 2 bimestre na turma 1A da materia de matematica


Thought: The current language of the user is: Portuguese. I need to use a tool to help me answer the question.
Action: ConsultarNotas
Action Input: {'params': AttributedDict([('turma', '1A'), ('bimestre', 2), ('disciplina', 'matematica'), ('aluno', 'ANNA CLARA CALERO GUIDA')])}
Observation: Nenhum aluno encontrado com os filtros aplicados: {'turma': '1A', 'alunos': 'ANNA CLARA CALERO GUIDA', 'bimestre': 2, 'nota': '6.1', 'disciplina': 'matematica'}
> Running step 9ac0ae12-e03f-4bb0-91f7-d0b3dfc17f0a. Step input: None
Thought: I didn't find the student's note with the applied filters. I need to inform the user that no student was found with the applied filters.
Answer: Nenhum aluno encontrado com os filtros aplicados.
> Running step 9b1d3a93-5e48-4795-a012-8d557d053fce. Step input: quero a nota do aluno ANNA CLARA CALERO GUID

Traceback (most recent call last):
  File "c:\Users\user\Documents\projetos\python\tcc\projeto\venv\Lib\site-packages\gradio\queueing.py", line 626, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\user\Documents\projetos\python\tcc\projeto\venv\Lib\site-packages\gradio\route_utils.py", line 350, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\user\Documents\projetos\python\tcc\projeto\venv\Lib\site-packages\gradio\blocks.py", line 2235, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\user\Documents\projetos\python\tcc\projeto\venv\Lib\site-packages\gradio\blocks.py", line 1746, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\user\Document

In [177]:
parametros_usados


{'turma': '1A',
 'alunos': 'ANNA CLARA CALERO GUIDA',
 'bimestre': 2,
 'nota': '6.1',
 'disciplina': 'matematica'}

In [112]:
novo1

[(<gradio.components.chatbot.Chatbot at 0x1fdd29088f0>,
  <gradio.components.state.State at 0x1fdd27c72c0>)]